# Use Weights & Biases Sweeps to automate hyperparameter search and explore the space of possible models.

### Overview:
1. Make training script

2. Define the sweep configuration

3. Initialize the Sweep

4. Start Sweep agents

## 1. Make training script

We define a main loop which mimics training and validation.

In [1]:
import random
import numpy as np

def train_one_epoch(epoch, lr, bs): 
  acc = 0.25 + ((epoch/30) +  (random.random()/10))
  loss = 0.2 + (1 - ((epoch-1)/10 +  random.random()/5))
  return acc, loss

def evaluate_one_epoch(epoch): 
  acc = 0.1 + ((epoch/20) +  (random.random()/10))
  loss = 0.25 + (1 - ((epoch-1)/10 +  random.random()/6))
  return acc, loss
  
config = {
    'lr' : 0.0001,
    'bs' : 16,
    'epochs': 5
}

def main():
  
    lr = config['lr']
    bs = config['bs']
    epochs = config['epochs']

    for epoch in np.arange(1, epochs):
      train_acc, train_loss = train_one_epoch(epoch, lr, bs)
      val_acc, val_loss = evaluate_one_epoch(epoch)
      
      print('epoch: ', epoch)
      print('training accuracy:', train_acc,'training loss:', train_loss)
      print('validation accuracy:', val_acc,'training loss:', val_loss)

In [2]:
main()

epoch:  1
training accuracy: 0.3113456825587058 training loss: 1.145971184647756
validation accuracy: 0.20672464098316373 training loss: 1.1247862864174993
epoch:  2
training accuracy: 0.33509600589836325 training loss: 0.9732455982264627
validation accuracy: 0.21646574674379152 training loss: 1.0685746625803234
epoch:  3
training accuracy: 0.3827244896966583 training loss: 0.9237935585435915
validation accuracy: 0.2598441394247611 training loss: 0.910645312819166
epoch:  4
training accuracy: 0.4167262793488241 training loss: 0.8679791699419928
validation accuracy: 0.38912418928719306 training loss: 0.9459146754093105


## 2. Define the sweep configuration

A Weights & Biases Sweep combines a strategy for exploring hyperparameter values with the code that evaluates them. The strategy can be as simple as trying every option or as complex as Bayesian Optimization and Hyperband (BOHB).

Define your strategy in the form of a sweep configuration. Specify the configuration either in a:

1. Python nested dictionary data structure if you use a Jupyter Notebook or Python script.

2. YAML file if you use the command line (CLI).

In [3]:
sweep_configuration = {
    'method': 'random',
    'name': 'sweep',
    'metric': {
        'goal': 'maximize', 
        'name': 'val_acc'
        },
    'parameters': {
        'bs': {'values': [16, 32, 64]},
        'epochs': {'values': [5, 10, 15]},
        'lr': {'max': 0.1, 'min': 0.0001}
     }
}

Suppose you want W&B Sweeps to maximize the validation accuracy during training. Within your Python script you store the validation accuracy in a variable `val_acc`.

In your YAML configuration file you define this as:

```
metric:
  goal: maximize
  name: val_acc
```

In addition, you must log the variable `val_acc` (in this example) within your Python script or Jupyter Notebook to W&B.

```
wandb.log({
        'val_acc': val_acc
      })
```

Read this document for more information about the sweep configuration structure, search methods, and config keys:

https://docs.wandb.ai/guides/sweeps/define-sweep-configuration

In [4]:
import wandb
import random
import numpy as np

def train_one_epoch(epoch, lr, bs): 
  acc = 0.25 + ((epoch/30) +  (random.random()/10))
  loss = 0.2 + (1 - ((epoch-1)/10 +  random.random()/5))
  return acc, loss

def evaluate_one_epoch(epoch): 
  acc = 0.1 + ((epoch/20) +  (random.random()/10))
  loss = 0.25 + (1 - ((epoch-1)/10 +  random.random()/6))
  return acc, loss
  
config = {
    'lr' : 0.0001,
    'bs' : 16,
    'epochs': 5
}

def main():
    run = wandb.init()
  
    # Note that we define values from `wandb.config` instead of 
    # defining hard values
    lr = config['lr']
    bs = config['bs']
    epochs = config['epochs']

    for epoch in np.arange(1, epochs):
      train_acc, train_loss = train_one_epoch(epoch, lr, bs)
      val_acc, val_loss = evaluate_one_epoch(epoch)
      
      wandb.log({
        'epoch': epoch, 
        'train_acc': train_acc,
        'train_loss': train_loss, 
        'val_acc': val_acc, 
        'val_loss': val_loss
      })

## 3. Initialize the Sweep

Weights & Biases uses a Sweep Controller to manage sweeps on the cloud (standard), locally (local) across one or more machines. After a run completes, the sweep controller will issue a new set of instructions describing a new run to execute. These instructions are picked up by agents who actually perform the runs. In a typical W&B Sweep, the controller lives on the Weights & Biases server. Agents live on your machine(s).

In [5]:
sweep_id = wandb.sweep(sweep=sweep_configuration, project="project-name")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: i09xbal4
Sweep URL: https://wandb.ai/paibl/project-name/sweeps/i09xbal4


The `wandb.sweep` function returns the sweep ID. The sweep ID includes the entity name and the project name. Make a note of the sweep ID.

## 4. Start Sweep agents

Start a W&B Sweep on one or more agents on one or more machines. W&B Sweep agents query the Weights & Biases server you launched when you initialized a W&B Sweep (wandb sweep) for hyperparameters and use them to run model training.

**Random and Bayesian searches will run forever. You must stop the process from the command line, within your python script, or the Sweeps UI.** 
If you want, pass an integer value to the count parameter to set the maximum number of runs to try.
 - i.e. `count = 3`

In [6]:
wandb.agent(sweep_id=sweep_id, function=main, count=3)

wandb: Agent Starting Run: 3zoql3mx with config:
wandb: 	bs: 16
wandb: 	epochs: 5
wandb: 	lr: 0.034613228344562284
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/home/eranario/miniconda3/envs/effdet/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: Currently logged in as: earlranario (paibl). Use `wandb login --relogin` to force relogin


epoch,▁▃▆█
train_acc,▃█▁▇
train_loss,█▄▃▁
val_acc,▁▄▃█
val_loss,█▇▆▁
epoch,4
train_acc,0.39254
train_loss,0.75048
val_acc,0.36565
val_loss,0.8212


wandb: Agent Starting Run: fvsyat9d with config:
wandb: 	bs: 32
wandb: 	epochs: 10
wandb: 	lr: 0.09042555293978376
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


epoch,▁▃▆█
train_acc,▁▆█▇
train_loss,▇█▁▁
val_acc,▂▁▆█
val_loss,█▆▄▁
epoch,4
train_acc,0.43539
train_loss,0.86436
val_acc,0.37954
val_loss,0.80203


wandb: Agent Starting Run: y5euo5he with config:
wandb: 	bs: 32
wandb: 	epochs: 15
wandb: 	lr: 0.08240068175073631
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


epoch,▁▃▆█
train_acc,▁▅▇█
train_loss,█▄▅▁
val_acc,▁▆██
val_loss,█▅▁▁
epoch,4
train_acc,0.47262
train_loss,0.81607
val_acc,0.33088
val_loss,0.90021
